# Missing Value Treatment

In [13]:
import numpy as np
import pandas as pd

## Load training and test data

In [14]:
# training dataframe and test dataframe
train_df = pd.read_csv('~/Documents/MIDS/W207/Final Project/data/train.csv')
test_df = pd.read_csv('~/Documents/MIDS/W207/Final Project/data/test.csv')

# Check for duplicates
print('Number of duplicate(s): %d\n' %train_df.duplicated().sum())

print('Training data shape:', train_df.shape)
print('Test data shape:', test_df.shape)

Number of duplicate(s): 0

Training data shape: (1460, 81)
Test data shape: (1459, 80)


## Check if features in train and test data are consistent

In [15]:
# Don't need Id column
train_df = train_df.drop(['Id'], axis=1)
test_df = test_df.drop(['Id'], axis=1)

#columns of train and test data
train_col = train_df.columns
test_col = test_df.columns

# Which feature is in train data but not test data?
diff_feat = np.setdiff1d(train_col, test_col)
print('Feature in train data but not test data:', diff_feat, '\n')

result = all(feature in train_col for feature in test_col)
if result:
    print('All features in test data are also features in training data')
else:
    print('Not all features in test data are also features in training data')

Feature in train data but not test data: ['SalePrice'] 

All features in test data are also features in training data


## Features with percentage of missing values greater than threshold

In [16]:
null_count = train_df.isnull().sum().sort_values(ascending=False)
null_percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
null_df = pd.concat([null_count, null_percent], axis=1, keys=['Total', 'Percent'])
null_df.head(20)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageType,81,0.055479
GarageCond,81,0.055479
GarageFinish,81,0.055479
GarageQual,81,0.055479


In [17]:
# features in train_df with percent of missing values greater than 80% (p_thres = 0.8)
p_thres = 0.8

# features with missing value percentage greater than threshold
del_ind = (null_df[null_df['Percent'] > p_thres]).index

## Function to treat missing values

In [18]:
# features with 'NA' as a valid entry
na_col = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']

def col_miss_replace(df, feature, check_feature):
    feature_mask = df[feature].isna()
    true_mask = df[check_feature].isin(['None'])

    false_mask = np.invert(true_mask)
    df[feature] = df[feature].mask(feature_mask & true_mask, 0)
    df[feature] = df[feature].mask(feature_mask & false_mask, df[feature].median())
    
def missing_val_treat(df):
    # deletes features in del_ind
    df.drop(del_ind, axis=1, inplace=True)
    
    # replace missing values with 'None' for features with 'NA' as valid entry
    df[na_col] = df[na_col].fillna('None')
    
    # features with missing vals that will take mode
    mode_col = ['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 
                    'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 
                    'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 
                    'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 
                    'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'PavedDrive', 
                    'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'YearBuilt']
    
    # features with missings that will take median
    med_col = ['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
                       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 
                       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 
                       'PoolArea', 'MiscVal']
    
    for col in mode_col:
        df[col].fillna(df[col].mode()[0], inplace=True)
        
    df[med_col] = df[med_col].fillna(df[med_col].median())
    
    bsmt_col = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']
    for col in bsmt_col:
        col_miss_replace(df, col, 'BsmtQual')
    
    col_miss_replace(df, 'MasVnrArea', 'MasVnrType')
    col_miss_replace(df, 'GarageCars', 'GarageCond')
    col_miss_replace(df, 'GarageArea', 'GarageCond')
    col_miss_replace(df, 'Fireplaces', 'FireplaceQu')
    
    df.loc[df['GarageYrBlt'] == 'None', 'YearBuilt'] = df['YearBuilt'].mode()[0]
    
    df['YearRemodAdd'].fillna(df['YearBuilt'], inplace=True)
    
    #what to fill garage year if no garage?
    df.loc[df['GarageCond'] == 'None', 'GarageYrBlt'] = df['YearBuilt']

In [19]:
missing_val_treat(train_df)
missing_val_treat(test_df)